In [66]:
import rdflib
from rdflib import RDF

In [62]:
class Property:
    def __init__(self, value, role, subject):
        self.value = value
        self.role = role
        self.subject = subject

class Definition:
    def __init__(self, wn_sense, definienda, properties):
        self.wn_sense = wn_sense
        self.properties = properties
        self.definienda = definienda

In [113]:
definition_query = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT DISTINCT ?definition
WHERE{
    ?definition rdf:type ?property .
    FILTER(?property != rdf:Statement)
}"""

property_template = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT ?value ?role ?subject
WHERE{{
<{}>      rdf:type      ?property .
?property rdf:object    ?value .
?property rdf:predicate ?role .
?property rdf:subject   ?subject
}}"""

nested_property_template = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT ?role ?subject ?subValue ?subRole ?subSubject
WHERE {{
<{}>             rdf:type    ?property .
?property        rdf:object    ?subProperty .
?property        rdf:predicate ?role .
?property        rdf:subject   ?subject .
?subProperty     rdf:object    ?subValue .
?subProperty     rdf:predicate ?subRole .
?subProperty     rdf:subject   ?subSubject
}}"""

label_query_template = """PREFIX rdf: <http://www.w3.org/2000/01/rdf-schema#>
SELECT DISTINCT ?definiendum
WHERE {{
<{}>     rdf:label    ?definiendum
}}
"""

In [4]:
wn_graph = rdflib.Graph()
wn_graph.parse("WN_DSR_model_XML.rdf")

definitions = wn_graph.query(definition_query)

In [129]:
wn_definitions = []
# Definitions consist of rows of results, each row being a tuple
# containing the URI of WN synset as its only element
for definition in definitions:
    current_properties = []
    current_definienda = []
    
    wn_uri = str(definition[0])
    has_nested_properties = False
    
    wn_definienda = wn_graph.query(label_query_template.format(wn_uri))
    for wn_definiendum in wn_definienda:
        current_definienda.append(wn_definiendum[0])
    wn_properties = wn_graph.query(property_template.format(wn_uri))
    # TODO: figure out how to represent nested properties
    for wn_property in wn_properties:
        if(type(wn_property[0]) == rdflib.term.Literal):
            value = wn_property[0]
            role = wn_property[1]
            subject = wn_property[2]
            current_properties.append(Property(value, role, subject))
        else:
            has_nested_properties = True
    wn_definitions.append(Definition(wn_uri, current_definienda, current_properties))

In [130]:
x_graph = rdflib.Graph()
efficient_cause = rdflib.term.URIRef('http://nlp/resources/AristotelianCauses#has_efficient_cause')
final_cause = rdflib.term.URIRef('http://nlp/resources/AristotelianCauses#has_final_cause')
rdf_schema = rdflib.Namespace("http://www.w3.org/2000/01/rdf-schema#")
for definition in wn_definitions:
    
    for definiendum in definition.definienda:
        x_graph.add( (rdflib.term.URIRef(definition.wn_sense), rdf_schema['label'], definiendum) )
    
    for wn_property in definition.properties:
        if wn_property.role == rdflib.term.URIRef('http://nlp/resources/DefinitionSemanticRoles#has_purpose'):
            x_graph.add( (rdflib.term.URIRef(definition.wn_sense), final_cause, wn_property.value) )
        elif wn_property.role == rdflib.term.URIRef('http://nlp/resources/DefinitionSemanticRoles#has_diff_event'):
            x_graph.add( (rdflib.term.URIRef(definition.wn_sense), efficient_cause, wn_property.value) )

In [131]:
with open("x_graph.n", "wb") as graph_file:
    graph_file.write(x_graph.serialize(format="n3"))